# Stepik ML contest

Мы начинаем соревнование! 

Задача нам уже знакома - нужно предсказать, сможет ли пользователь успешно закончить онлайн курс Анализ данных в R.
Мы будем считать, что пользователь успешно закончил курс, если он правильно решил больше 40 практических заданий.

В данных:

submission_data_test.csv

events_data_test.csv

хранится информация о решениях и действиях для 6184 студентов за первые два дня прохождения курса. Это 6184 студентов, которые проходили курс в период с мая 2018 по январь 2019.   

Используя данные о первых двух днях активности на курсе вам нужно предсказать, наберет ли пользователь более 40 баллов на курсе или нет.

В этих данных, вам доступны только первые дня активности студентов для того, чтобы сделать предсказание. На самом деле, используя эти данные, вы уже можете сделать прогноз. Например, если пользователь за первые два дня набрал 40 баллов, скорее всего он наберет более 40 баллов в дальнейшем. Чтобы подкрепить такие гипотезы, вы можете использовать данные, на которые мы исследовали в первых двух модулях курса, где для всех пользователей представлены все данные об их активности на курсе. 

In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [23]:
event_data_train = pd.read_csv('event_data_train.csv')

In [24]:
submissions_data_train = pd.read_csv('submissions_data_train.csv')

In [25]:
def time_filter(data : object, days = 2) -> object:
    
    """  Функция создает датафрэйм с данными по действиям юзера за первых 2 дня его прибывания на Stepik"""
    
    # преобразовываем days в секунды
    learning_time_threshold = days * 24 * 60 * 60
    
    # группируем все данные по user_id и определяем у каждого уникального user_id минимальный timestamp 
    # (первый день на Stepik) и записываем данные в переменную
    min_user_time_submissions = data.groupby('user_id')\
                            .agg({'timestamp': 'min'}) \
                            .rename(columns={'timestamp': 'min_timestamp'}) \
                            .reset_index()
    
    # добавляем данные о первом дне в новый датафрэйм
    data_two_days = data.merge(min_user_time_submissions, on='user_id', how='outer')
    
    # отбираем все даннные, у которых timestamp <= min_timestamp + 2 дня
    data_two_days = data_two_days.query("timestamp <= min_timestamp + @learning_time_threshold")
    
    # удаляем столбец min_timestamp
    data_two_days = data_two_days.drop(['min_timestamp'], axis=1)
    
    # проверка
    assert min_user_time_submissions.user_id.nunique() == data_two_days.user_id.nunique(), 'Кол-во уникальных рользователей не совпадает'
    
    return data_two_days

In [26]:
def count_action_submissions(event_data_two_days: object, submissions_data_two_days: object)->object:
    
    """ Создает датафрэйм с основными фичами"""
    
    # Сводная таблица по двум дням для event_data
    user_event_data_two_days = event_data_two_days.pivot_table(index = 'user_id', 
                                                                       columns='action', 
                                                                       values = 'step_id', 
                                                                       aggfunc = 'count', 
                                                                       fill_value = 0) \
                                                                       .reset_index()
    
    
    # Сводная таблица по двум дням для submissions_data
    user_submissions_data_two_days = submissions_data_two_days.pivot_table(index = 'user_id',
                                                                                   columns='submission_status',
                                                                                   values = 'step_id',
                                                                                   aggfunc = 'count',
                                                                                   fill_value = 0).reset_index()
    
    #  Соединяем два дата фрэйма                                                                             
    user_data = pd.merge(user_submissions_data_two_days, user_event_data_two_days, on = 'user_id', how = 'outer').fillna(0)
    
    # Проверка
    assert user_data.user_id.nunique() == event_data_two_days.user_id.nunique(), \
                       'Не совпадает кол-во user_id в event_data_two_days и user_event_data_two_days'
   
    return user_data

In [27]:
def count_attempts(submissions_data_two_days: object) -> object:
    
    """ Функция подсчитывает кол-во всех попыток usera"""
    
    # Считаем кол-во попыток для  user_id
    count_attempts = submissions_data_two_days.groupby('user_id').agg({"step_id":"count"})\
                                                              .rename(columns={"step_id": "count_attempts"}).reset_index()
    
    # Проверка
    assert count_attempts.user_id.nunique() == submissions_data_two_days.user_id.nunique(), \
                                'Не совпадает кол-во user_id в submissions_data_two_days и count_attempts'
    
    return count_attempts

In [28]:
def count_unique_attempts(submissions_data_two_days: object) -> object:
    
    """ Функция подсчитывает кол-во всех УНИКАЛЬНЫХ попыток usera"""
    
    # Считаем кол-во уникальных  попыток для  user_id
    count_unique_attempts = submissions_data_two_days.groupby('user_id').step_id.nunique().to_frame().reset_index() \
                                       .rename(columns={'step_id': 'count_unique_attempts'})
    
    # Проверка
    assert count_unique_attempts.user_id.nunique() == submissions_data_two_days.user_id.nunique(), \
                                'Не совпадает кол-во user_id в submissions_data_two_days и count_unique_attempts'
    
    return count_unique_attempts

In [29]:
def users_count_correct(submissions_data_train : object, raiting = 40) -> object:
    
    """ Создает переменную, которую будем предсказывать"""
    
    # Считаем кол-во правильно решенных задач
    users_count_correct = submissions_data_train[submissions_data_train.submission_status == 'correct'] \
                .groupby('user_id').agg({'step_id': 'count'}) \
                .reset_index().rename(columns={'step_id': 'corrects'})
    
    # Проверяем количесвто правильно решенных задач, если больше raiting, то True, иначе False. Добавляем колонку с данными
    users_count_correct['passed_course'] = (users_count_correct.corrects >= raiting).astype('int')
    
    # удаляем колонку corrects
    users_count_correct = users_count_correct.drop(['corrects'], axis=1)
       
    return users_count_correct

In [30]:
def creat_df_train(event_data_train: object, submissions_data_train: object, days = 2) -> object:
    
    # Определяем дейсвтия и события user_id за два дня
    event_data_train_two_days = time_filter(event_data_train, days)
    submissions_data_train_two_days = time_filter(submissions_data_train, days)
    
    # Создаем датафрэйм с основными фичами
    df_train = count_action_submissions(event_data_train_two_days, submissions_data_train_two_days)
    
    # Подсчитывает кол-во всех попыток usera
    attempts = count_attempts(submissions_data_train_two_days)
    
    # Подсчитывает кол-во всех УНИКАЛЬНЫХ попыток usera
    unique_attempts = count_unique_attempts(submissions_data_train_two_days)
    
    # Создаем переменную, которую будем предсказывать
    count_correct = users_count_correct(submissions_data_train)
    
    # Соединяем все полученные данные
    df_train = df_train.merge(attempts, on = 'user_id', how = 'outer')
    df_train = df_train.merge(unique_attempts, on = 'user_id', how = 'outer')
    df_train = df_train.merge(count_correct, on = 'user_id', how = 'outer').fillna(0)
    

    return df_train

In [31]:
def creat_df_test(event_data_test: object, submissions_data_test: object, days = 2) -> object:
    
    # Определяем дейсвтия и события user_id за два дня
    event_event_data_test = time_filter(event_data_test, days)
    submissions_submissions_data_test = time_filter(submissions_data_test, days)
    
    # Создаем датафрэйм с основными фичами
    df_test = count_action_submissions(event_data_test, submissions_data_test)
    
    # Подсчитывает кол-во всех попыток usera
    attempts = count_attempts(submissions_data_test)
    
    # Подсчитывает кол-во всех УНИКАЛЬНЫХ попыток usera
    unique_attempts = count_unique_attempts(submissions_data_test)
    
    # Создаем переменную, которую будем предсказывать
    #count_correct = users_count_correct(submissions_data_train)
    
    # Соединяем все полученные данные
    df_test = df_test.merge(attempts, on = 'user_id', how = 'outer')
    df_test = df_test.merge(unique_attempts, on = 'user_id', how = 'outer').fillna(0)
    #df_test = df_test.merge(count_correct, on = 'user_id', how = 'outer').fillna(0)
    
    return df_test

In [32]:
event_data_train = pd.read_csv('event_data_train.csv')
submissions_data_train = pd.read_csv('submissions_data_train.csv')

In [33]:
data_train = creat_df_train(event_data_train, submissions_data_train)

In [34]:
y = data_train['passed_course'].map(int)
X_data_train = data_train.drop(['passed_course'], axis=1)

In [35]:
X_data_train

,user_id,correct,wrong,discovered,passed,started_attempt,viewed,count_attempts,count_unique_attempts
0,2,2.0,0.0,9,9,2,9,2.0,2.0
1,3,4.0,4.0,15,15,4,20,8.0,4.0
2,5,2.0,2.0,1,1,0,1,4.0,2.0
3,8,9.0,21.0,109,84,37,154,30.0,11.0
4,14,0.0,1.0,4,3,1,9,1.0,1.0
...,...,...,...,...,...,...,...,...,...
19229,26773,0.0,0.0,1,1,0,1,0.0,0.0
19230,26774,0.0,0.0,1,1,0,1,0.0,0.0
19231,26788,0.0,0.0,1,1,0,1,0.0,0.0
19232,26789,0.0,0.0,2,2,0,2,0.0,0.0


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_data_train, y, test_size=0.2, random_state=42)
    
param_grid = {'randomforestclassifier__n_estimators': range(20, 51, 3), 
                  'randomforestclassifier__max_depth': range(5, 14)}
    
pipe = make_pipeline(RandomForestClassifier())
pipe.fit(X_train, y_train)
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
print(f"Наилучшие параметры: {grid.best_params_}")
    
ypred_prob = grid.predict_proba(X_test)
    
roc_score = roc_auc_score(y_test, ypred_prob[:, 1])
score = grid.score(X_test, y_test)
print(f"Правильность на тестовом наборе: {score:.2f}")
print(roc_score)

Наилучшие параметры: {'randomforestclassifier__max_depth': 5, 'randomforestclassifier__n_estimators': 29}
Правильность на тестовом наборе: 0.90
0.8872076426280012


In [37]:
submission_data_test = pd.read_csv('https://stepik.org/media/attachments/course/4852/submission_data_test.csv')
events_data_test = pd.read_csv('https://stepik.org/media/attachments/course/4852/events_data_test.csv')

In [42]:
df_test = creat_df_test(events_data_test, submission_data_test).fillna(0)  

In [45]:
test_data = df_test
    
X_train, X_test, y_train, y_test = train_test_split(X_data_train, y, test_size=0.2, random_state=42)
    
pipe = make_pipeline(RandomForestClassifier(max_depth=7, n_estimators=40,  random_state=42))
pipe.fit(X_train, y_train)
    
ypred_prob = pipe.predict_proba(X_test)
    
roc_score = roc_auc_score(y_test, ypred_prob[:, 1])
score = pipe.score(X_test, y_test)
print(f"Правильность на валид наборе: {score:.3f}")
print(f"Roc_auc_score на валид наборе: {roc_score:.5f}")
    
ypred_prob_final = pipe.predict_proba(test_data)
result = test_data['user_id'].to_frame()
result['is_gone'] = ypred_prob_final[:, 1]
result[['user_id', 'is_gone']].to_csv(f'my_predict_{roc_score:.5f}.csv', index=False)
print(f'Результы записанны в файл my_predict_{roc_score:.5f}.csv')

Правильность на валид наборе: 0.900
Roc_auc_score на валид наборе: 0.88460
Результы записанны в файл my_predict_0.88460.csv
